In [6]:
import os
import numpy as np
from PIL import Image
import glob

In [7]:
path = "Composed_Data/"
files = os.listdir(path)

In [8]:
def read_and_separate_image_pair(file):
    image = np.asarray(Image.open(file))
    w = image.shape[1] // 2
    im1 = Image.fromarray(image[:, w:, :])
    im2 = Image.fromarray(image[:, :w, :])
    return im1, im2

### Source: https://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders
def rotated_rect_with_max_area(w, h, angle):
    """
    Given a rectangle of size w x h that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle (maximal area) within the rotated rectangle.
    """
    if w <= 0 or h <= 0:
        return 0, 0
    
    width_is_longer = w >= h
    side_long, side_short = (w,h) if width_is_longer else (h,w)
    
    # since the solutions for angle, -angle and 180-angle are all the same,
    # if suffices to look at the first quadrant and the absolute values of sin,cos:
    sin_a, cos_a = abs(np.sin(angle)), abs(np.cos(angle))
    if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
        # half constrained case: two crop corners touch the longer side,
        #   the other two corners are on the mid-line parallel to the longer line
        x = 0.5*side_short
        wr,hr = (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
    else:
        # fully constrained case: crop touches all 4 sides
        cos_2a = cos_a*cos_a - sin_a*sin_a
        wr,hr = (w*cos_a - h*sin_a)/cos_2a, (h*cos_a - w*sin_a)/cos_2a
    
    return wr, hr

def random_rotate(im1, im2, max_angle=20):
    h, w = im1.size
    angle = 2 * max_angle * np.random.rand() - max_angle
    wr, hr = rotated_rect_with_max_area(w, h, np.deg2rad(angle))
    x = np.ceil((w - wr) / 2)
    y = np.ceil((h - hr) / 2)
    shrink = (x, y, w - x, h - y)
    im1 = im1.rotate(angle, Image.NEAREST, expand=0).crop(shrink).resize(im1.size)
    im2 = im2.rotate(angle, Image.NEAREST, expand=0).crop(shrink).resize(im2.size)
    return im1, im2

def random_zoom_and_crop(im1, im2, half_max_crop=0.25):
    left = round(half_max_crop * im1.size[0] * np.random.rand())
    right = round(im1.size[0] - half_max_crop * im1.size[0] * np.random.rand())
    top = round(half_max_crop * im1.size[1] * np.random.rand())
    bottom = round(im1.size[1] - half_max_crop * im1.size[1] * np.random.rand())
    im1 = im1.crop((left, top, right, bottom)).resize(im1.size)
    im2 = im2.crop((left, top, right, bottom)).resize(im2.size)
    return im1, im2

def horizontal_concatenation(im1, im2):
    paired = Image.new('RGB', (im1.width + im2.width, im1.height))
    paired.paste(im2, (0, 0))
    paired.paste(im1, (im2.width, 0))
    return paired

In [9]:
num_of_copies = 1

for file in files:
    if '.jpg' in file:
        for num in range(num_of_copies):
            im1, im2 = read_and_separate_image_pair(os.path.join(path, file))
            im1, im2 = random_rotate(im1, im2)
            im1, im2 = random_zoom_and_crop(im1, im2)
            paired = horizontal_concatenation(im1, im2)
            paired.save(f"{path}{file[:-4]}_modified{num}.jpg")